In [2]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils

datagenerator = DataGenerator()  # data generator
utils = Utils()  # utils function
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random

random.seed(42)

In [3]:
# dataset and model list / dict
dataset_list = [
    "01_ALOI",
    "02_annthyroid",
    "03_backdoor",
    "04_breastw",
    "05_campaign",
    "06_cardio",
    "07_Cardiotocography",
    "08_celeba",
    "09_census",
    "99_linear",
    "99_circles",
    "99_moons",
    "99_clusters",
    "99_oilflow",
]
dataset = dataset_list[2]
datagenerator.dataset = dataset

data = datagenerator.generator(
    la=0.999,
    realistic_synthetic_mode=None,
    stdscale=True,
    minmax=False,
    noise_type=None,
)

Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
lb_test = torch.tensor(data["y_test"], dtype=torch.float32)
idx_n = np.where(lb_train == 0)[0]
Y_train = Y_train[idx_n]
lb_train = lb_train[idx_n]

subsampling for dataset 03_backdoor...
current noise type: None
{'Samples': 10000, 'Features': 196, 'Anomalies': 236, 'Anomalies Ratio(%)': 2.36}


In [4]:
n_train = len(Y_train)
data_dim = Y_train.shape[1]
latent_dim = 15
n_inducing = 50
n_epochs = 1000
nn_layers = 15, 15
lr = 0.01
batch_size = 128

In [5]:
gplvm = AD_GPLVM(latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size)

In [6]:
gplvm.fit(Y_train)

Setando Kernel RBF Padrão


Loss: 268.62, iter no: 18:   2%|▏         | 18/1000 [00:03<02:57,  5.53it/s]


KeyboardInterrupt: 

In [8]:
score = gplvm.predict_score(Y_test)
score

array([[0.01765113],
       [0.00413254],
       [0.00457133],
       ...,
       [0.00419686],
       [0.00084705],
       [0.00442858]], dtype=float32)

In [ ]:
utils.metric(y_true=lb_test, y_score=predict_score(Y_test))

In [9]:
utils.metric(y_true=lb_test, y_score=score)

{'aucroc': 0.7884102154751657, 'aucpr': 0.29993227027469993}